In [ ]:
%pylab inline
import calin.math.geometry
import calin.math.hex_array
import calin.simulation.vs_optics
import calin.simulation.ray_processor

In [ ]:
def dms(d,m,s):
    # Note this function fails for "negative" d=0 (e.g. -00:30:00)
    sign = 1
    if(d<0):
        sign = -1
        d = abs(d)
    return sign * (d + m/60.0 + s/3600.0)

mst = calin.ix.simulation.vs_optics.IsotropicDCArrayParameters()
mst.mutable_array_origin().set_latitude(dms(28, 45, 47.36))
mst.mutable_array_origin().set_longitude(dms(-17, 53, 23.93))
mst.mutable_array_origin().set_elevation(2147 * 100.0)
scope = mst.mutable_prescribed_array_layout().add_scope_positions();
scope.set_z(mst.array_origin().elevation())
mst.mutable_reflector_frame().set_optic_axis_rotation(-90);
dc = mst.mutable_reflector()
dc.set_curvature_radius(1920)
dc.set_aperture(1230)
dc.set_facet_num_hex_rings(5)
dc.mutable_psf_align().set_object_plane(inf) # 10 * 1e5);
dc.set_alignment_image_plane(1600)
dc.set_facet_spacing(122)
dc.set_facet_size(120)
dc.set_facet_focal_length(1607)
dc.set_facet_focal_length_dispersion(1)
dc.set_facet_spot_size_probability(0.8)
dc.set_facet_spot_size(0.5 * 2.8) # Spot size of 28mm at 2F
dc.set_facet_spot_size_dispersion(0.5 * 0.02)
dc.set_facet_labeling_parity(True)
dc.set_weathering_factor(1.0)
for id in [1,62,67,72,77,82,87]: dc.add_facet_missing_list(id-1)
mst.mutable_focal_plane().set_camera_diameter(235)
mst.mutable_focal_plane().mutable_translation().set_y(1/(1.0/dc.alignment_image_plane()-1/(10 * 1e5)))
mst.mutable_pixel().set_spacing(5)
mst.mutable_pixel().set_cone_inner_diameter(5)
mst.mutable_pixel().set_cone_survival_prob(1)
mst.mutable_pixel().set_hex_module_size(1)
mst.mutable_pixel().set_module_num_hex_rings(9)
u1,v1 = calin.math.hex_array.cluster_hexid_to_center_uv(1,1)
x1,y1 = calin.math.hex_array.uv_to_xy(u1,v1)
rot = arctan2(-y1,x1)/pi*180 - 30
mst.mutable_pixel().set_grid_rotation(rot)

obs_camera_box = mst.add_obscurations()
obs_camera_box.aligned_box().max_corner().set_x(150)
obs_camera_box.aligned_box().max_corner().set_y(mst.focal_plane().translation().y()+150)
obs_camera_box.aligned_box().max_corner().set_z(150)
obs_camera_box.aligned_box().min_corner().set_x(-150)
obs_camera_box.aligned_box().min_corner().set_y(mst.focal_plane().translation().y())
obs_camera_box.aligned_box().min_corner().set_z(-150)
obs_camera_box.aligned_box().set_incoming_only(True)

rng = calin.math.rng.RNG()
cta = calin.simulation.vs_optics.VSOArray()
cta.generateFromArrayParameters(mst, rng)

cta.pointTelescopesAzEl(0,90.0/180.0*pi);

scope = cta.telescope(0)

In [ ]:
pe_imager = calin.simulation.ray_processor.TelescopePSFCalcPEProcessor()
processor = calin.simulation.ray_processor.VSORayProcessor(cta, pe_imager, rng)

In [ ]:
dx = 1 # cm
dtheta = 0.1 / 180*pi # rad

fov_2 = 4.5 / 180.0*pi # rad
R = scope.reflectorIP()/2.0

x00 = scope.reflectorIPCenter()
scope.reflectorToGlobal_pos(x00)

all_theta = []
mean_x = []
mean_y = []
psf_x = []
psf_y = []

for theta in frange(0,fov_2,dtheta):
    pos_gen = calin.math.ray_generator.HexGridPlanePositionGenerator(R, dx)
    dir_gen = calin.math.ray_generator.SingleDirectionGenerator()
    ray_gen = calin.math.ray_generator.PositionNestedByDirectionRayGenerator(x00,
        calin.math.geometry.rotation_theta_phi(pi-theta,0), asarray([0,0,-100e5]), dir_gen, pos_gen)
    pe_imager.clear()
    processor.process_all_from_ray_generator(ray_gen)
    all_theta.append(theta/pi*180)
    mean_x.append(pe_imager.mom().mean_x())
    mean_y.append(pe_imager.mom().mean_y())
    psf_x.append(sqrt(pe_imager.mom().var_x()))
    psf_y.append(sqrt(pe_imager.mom().var_y()))

In [ ]:
plot(all_theta, asarray(psf_x)/scope.focalPlanePosition()[1]/pi*180, label='Tangential')
hold('on')
plot(all_theta, asarray(psf_y)/scope.focalPlanePosition()[1]/pi*180, label='Saggital')
legend(loc=2)
xlabel('Field angle [deg]')
ylabel('PSF RMS [deg]')
a=list(axis())
a[2] = 0
axis(a);

In [ ]:
plot(all_theta, asarray(mean_x)/scope.focalPlanePosition()[1]/pi*180, label='Tangential')
hold('on')
plot(all_theta, asarray(mean_y)/scope.focalPlanePosition()[1]/pi*180, label='Saggital')
legend(loc=2)
xlabel('Field angle [deg]')
ylabel('Image centroid location [deg]')
grid()
a=list(axis())
a[2] = 0
axis(a);